Instructions:

Implement Bayesian inference
1) Randomly generate n=100 (2D) random samples from a MVN with mean [-1, 1]; covariance [2, 1.3; 1.3; 4] 
2) Use Gibbs sampling to infer unknown parameters : mean & covariance

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

# 1 Generate Data from MVN

In [26]:
random_seed = 123
rng = np.random.default_rng(random_seed)

In [52]:
μ = np.array([-1, 1])
Σ = np.array([[2, 1.3], [1.3, 4]])
X = np.random.multivariate_normal(μ, Σ, size=100)

# 2 Bayesian Inference Setup

Posterior distribution (Normal Inverse Wishart) of $\mu$ and $\Sigma$

$$\begin{align}

    p(\mu,\Sigma | D) &= \mathcal{N}(\mu|\mathbf{m}_N, \frac{1}{\kappa_N}\Sigma) \times \mathcal{IW}(\Sigma,\mathbf{S}_N, \nu_N)\\ &= \mathcal{NIW}(\mu,\Sigma | \mathbf{m}_N, \kappa_N, \nu_N, \mathbf{S}_N)\\\\

    m_N &= \frac{\kappa_0\mathbf{m}_0 + N\bar{\mathbf{x}}}{\kappa_0 + N} \\
    \kappa_N &= \kappa_0 + N\\
    \nu_N &= \nu_0 + N \\
    \mathbf{S}_N &= \mathbf{S}_0 + \mathbf{S}_{\bar{\mathbf{x}}} + \frac{\kappa_0N}{\kappa_0 + N}(\bar{\mathbf{x}} - \mathbf{m}_0)(\bar{\mathbf{x}} - \mathbf{m}_0)^T\\
    &= \mathbf{S}_0 + \mathbf{S} + \kappa_0\mathbf{m}_0\mathbf{m}_0^T - \kappa_N\mathbf{m}_N\mathbf{m}_N^T \\

    \mathbf{S} &= \sum_{i=1}^{N} \mathbf{x}_i\mathbf{x}_i^T
    
\end{align}$$

Bartlett Decomposition for Sampling from Inverse-Wishart





In [ ]:
def gibbs(X, num_iters, num_burn, eps=1e-14):
    n,d = X.shape

    x_bar = X.mean(axis=0)

    μ_samples = np.zeros((num_iters + num_burn, d))
    Σ_samples = np.zeros((num_iters + num_burn, d, d))

    k_0 = 0.01
    k_n = k_0 + n

    μ_0 = x_bar.copy()

    ν_0 = d + 2 
    S_0 = np.eye(d)

    # Initialize
    Σ_samples[0] = np.cov(X.T)
    μ_samples[0] = x_bar.copy()

    for i in range(1, num_iters + num_burn):
        # Sample μ
        μ_n = (k_0 * μ_0 + n * x_bar) / k_n
        Σ_n = Σ_samples[i-1]/k_n
        μ = np.random.multivariate_normal(mean=μ_n, cov=Σ_n)

        # Sample Σ
        # S = np.sum([(x - μ).reshape(-1, 1) @ (x - μ).reshape(1, -1) for x in X], axis=0)
        S = sum(x @ x.T for x in X)
        # diff = (x_bar - μ_0).reshape(-1, 1)
        # S_n = S_0 + S + (k_0 * n) / k_n * (diff @ diff.T)
        S_n = S_0 + S + (k_0 * (μ_0 @ μ_0.T)) - (k_n * (μ_n @ μ_n.T))
        ν_n = ν_0 + n

        # Sample from Inverse-Wishart using Bartlett decomposition
        T = np.zeros((d, d))
        for i in range(d):
            T[i, i] = np.sqrt(np.random.chisquare(ν_n - i))
            for j in range(i):
                T[i, j] = np.random.normal()

        A = T @ T.T
        L = np.linalg.cholesky(np.linalg.inv(S_n))
        W = L @ A @ L.T
        Sigma = np.linalg.inv(W)

        # Store
        μ_samples[i] = μ
        Σ_samples[i] = Sigma

    return μ_samples[num_burn:], Σ_samples[num_burn:]



In [66]:
mus, sigmas = gibbs(X,1000, 100)

In [67]:
print(mus[-1], sigmas[-10])

[0. 0.] [[0. 0.]
 [0. 0.]]
